In [1]:
import numpy as np
import pandas as pd

In [2]:
import gspread

In [3]:
movies=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ1SF1Vm9dUJvVVMMyANFq8KKZOjc9oIbPCxIQKZs5dCB8XAbwHwQuJx9eVWoM1i-pdPXoUP2P2ubgr/pub?gid=444641845&single=true&output=csv')
credits=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vRCGybg-wnFfdvFuzGepvS2JDt16HgbZMPZa175D6h-zPSUqQsYLgQ8A_HkZESQqiisInJYTFs2LE98/pub?gid=834367730&single=true&output=csv')


In [4]:
movies=movies.merge(credits, on ='title')
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4787 entries, 0 to 4786
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4787 non-null   int64  
 1   genres                4787 non-null   object 
 2   homepage              1707 non-null   object 
 3   id                    4787 non-null   int64  
 4   keywords              4787 non-null   object 
 5   original_language     4787 non-null   object 
 6   original_title        4787 non-null   object 
 7   overview              4783 non-null   object 
 8   popularity            4787 non-null   float64
 9   production_companies  4787 non-null   object 
 10  production_countries  4787 non-null   object 
 11  release_date          4786 non-null   object 
 12  revenue               4787 non-null   int64  
 13  runtime               4785 non-null   float64
 14  spoken_languages      4787 non-null   object 
 15  status               

In [5]:
##keeping relevant data 

In [6]:
movies=movies[['id','title','overview','genres','keywords','cast','crew']]

In [7]:
##preprocessing

In [8]:
movies.isnull().sum()

id          0
title       0
overview    4
genres      0
keywords    0
cast        0
crew        1
dtype: int64

In [9]:
movies.dropna(inplace=True)

In [10]:
movies.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [11]:
movies.duplicated().head

<bound method NDFrame.head of 0       False
1       False
2       False
3       False
4       False
        ...  
4782    False
4783    False
4784    False
4785    False
4786    False
Length: 4782, dtype: bool>

In [12]:
movies.loc[movies.duplicated(),:]

,id,title,overview,genres,keywords,cast,crew


In [13]:
movies=movies.drop_duplicates()

In [14]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4782 entries, 0 to 4786
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        4782 non-null   int64 
 1   title     4782 non-null   object
 2   overview  4782 non-null   object
 3   genres    4782 non-null   object
 4   keywords  4782 non-null   object
 5   cast      4782 non-null   object
 6   crew      4782 non-null   object
dtypes: int64(1), object(6)
memory usage: 298.9+ KB


In [15]:
movies.shape

(4782, 7)

In [16]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [17]:
movies.iloc[0,3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [18]:
import ast
def convert(obj):
  Lgenre=[]
  for i in ast.literal_eval(obj):
    Lgenre.append(i['name'])
  return Lgenre


In [19]:
movies['genres']=movies['genres'].apply(convert)

In [20]:
movies['keywords']=movies['keywords'].apply(convert)

In [21]:
movies.iloc[0].cast

'[{"cast_id": 242, "character": "Jake Sully", "credit_id": "5602a8a7c3a3685532001c9a", "gender": 2, "id": 65731, "name": "Sam Worthington", "order": 0}, {"cast_id": 3, "character": "Neytiri", "credit_id": "52fe48009251416c750ac9cb", "gender": 1, "id": 8691, "name": "Zoe Saldana", "order": 1}, {"cast_id": 25, "character": "Dr. Grace Augustine", "credit_id": "52fe48009251416c750aca39", "gender": 1, "id": 10205, "name": "Sigourney Weaver", "order": 2}, {"cast_id": 4, "character": "Col. Quaritch", "credit_id": "52fe48009251416c750ac9cf", "gender": 2, "id": 32747, "name": "Stephen Lang", "order": 3}, {"cast_id": 5, "character": "Trudy Chacon", "credit_id": "52fe48009251416c750ac9d3", "gender": 1, "id": 17647, "name": "Michelle Rodriguez", "order": 4}, {"cast_id": 8, "character": "Selfridge", "credit_id": "52fe48009251416c750ac9e1", "gender": 2, "id": 1771, "name": "Giovanni Ribisi", "order": 5}, {"cast_id": 7, "character": "Norm Spellman", "credit_id": "52fe48009251416c750ac9dd", "gender": 

In [22]:
def convertcast(obj):
  Lcast=[]
  for i in ast.literal_eval(obj):
    counter = 0
    if counter != 3:
      Lcast.append(i['name'])
      counter+=1
    else:
      break
  return Lcast

In [23]:
movies['cast']=movies['cast'].apply(convertcast)


In [24]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [25]:
def convertcrew(obj):
  Lcrew=[]
  for i in ast.literal_eval(obj):
    if i['job']=='Director':
      Lcrew.append(i['name'])
      break
  return Lcrew

In [26]:
movies['crew']=movies['crew'].apply(convertcrew)

In [27]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]


In [28]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [29]:
movies

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley, ...",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux, R...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman, A...",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton,...",[Andrew Stanton]
...,...,...,...,...,...,...,...
4782,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...","[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",[Robert Rodriguez]
4783,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...","[Comedy, Romance]",[],"[Edward Burns, Kerry Bishé, Marsha Dietlein, C...",[Edward Burns]
4784,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,...","[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","[Eric Mabius, Kristin Booth, Crystal Lowe, Geo...",[Scott Smith]
4785,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is...",[],[],"[Daniel Henney, Eliza Coupe, Bill Paxton, Alan...",[Daniel Hsia]


In [30]:
movies['overview']=movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [31]:
movies

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[culture clash, future, space war, space colon...","[SamWorthington, ZoeSaldana, SigourneyWeaver, ...",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[JohnnyDepp, OrlandoBloom, KeiraKnightley, Ste...",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[DanielCraig, ChristophWaltz, LéaSeydoux, Ralp...",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[ChristianBale, MichaelCaine, GaryOldman, Anne...",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[based on novel, mars, medallion, space travel...","[TaylorKitsch, LynnCollins, SamanthaMorton, Wi...",[AndrewStanton]
...,...,...,...,...,...,...,...
4782,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui...","[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[CarlosGallardo, JaimedeHoyos, PeterMarquardt,...",[RobertRodriguez]
4783,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended...","[Comedy, Romance]",[],"[EdwardBurns, KerryBishé, MarshaDietlein, Cait...",[EdwardBurns]
4784,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,...","[Comedy, Drama, Romance, TVMovie]","[date, love at first sight, narration, investi...","[EricMabius, KristinBooth, CrystalLowe, GeoffG...",[ScottSmith]
4785,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is...",[],[],"[DanielHenney, ElizaCoupe, BillPaxton, AlanRuc...",[DanielHsia]


In [32]:
movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [33]:
newmovies=movies[['id','title','tags']]

In [34]:
newmovies

,id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4782,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4783,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4784,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4785,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [35]:
newmovies['tags']=newmovies['tags'].apply(lambda x:" ".join(x))

<ipython-input-35-ec19f40473f0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newmovies['tags']=newmovies['tags'].apply(lambda x:" ".join(x))


In [36]:
newmovies['tags']=newmovies['tags'].apply(lambda x:x.lower())

<ipython-input-36-26cb00c24070>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newmovies['tags']=newmovies['tags'].apply(lambda x:x.lower())


In [37]:
newmovies

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4782,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4783,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4784,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4785,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [38]:
##Preprocessing done

In [39]:
##stemming of tags

In [40]:
import nltk

In [41]:
from nltk.stem.porter import PorterStemmer
PS = PorterStemmer()

In [42]:
def stem(text):
  y = []
  for i in text.split():
    y.append(PS.stem(i))
  return " ".join(y)

In [43]:
newmovies['tags'] = newmovies['tags'].apply(stem)

<ipython-input-43-2f3a685faaa6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newmovies['tags'] = newmovies['tags'].apply(stem)


In [44]:
##vectorization of words

In [45]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(max_features=5000,stop_words='english')

In [46]:
movievectors = CV.fit_transform(newmovies['tags']).toarray()

In [47]:
movievectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [48]:
CV.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

In [49]:
## findinf simirality between movies and sorting them according to similarity 

In [50]:
from sklearn.metrics.pairwise import cosine_similarity as CS


In [51]:
moviesimilarity = CS(movievectors)

In [52]:
def recommend(movie):
  movie_index=newmovies[newmovies['title'] == movie].index[0]
  distances = moviesimilarity[movie_index]
  movie_list=sorted(list(enumerate(distances)),reverse =True, key = lambda x:x[1])[1:6]      ##enumerate is used so that index in not lost in sorting  
  for i in movie_list:
    print(newmovies.iloc[i[0]].title)

In [53]:
## Enter Movie for recommendations

In [54]:
recommend("")

IndexError: ignored